In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from statsmodels.tsa.ar_model import AutoReg
import math
import plotly.express as px
import plotly.graph_objs as go
import os

path = os.path.dirname(os.getcwd())

# Figure 1. Paradoxical relationship between IGE and wealth dynamics.

In [27]:
def simulate_wealth(surnames, W0_values, mu_values, beta_values, times, groups):
    records = []
    for surname, W0, mu, beta, group in zip(surnames, W0_values, mu_values, beta_values, groups):
        for t in times:
            if t == 0:
                wealth = W0
            else:
                exp_part = (1 - beta**t) * mu
                wealth = np.exp(exp_part) * W0**(beta**t)
            log_wealth = np.log(wealth)
            log_random = np.random.normal(loc=0, scale=1)
            records.append({'Surname': surname, 
                            'Group': group,
                            'Time': t, 
                            'Wealth': wealth, 
                            'log_Wealth': log_wealth, 
                            'mu': mu, 
                            'beta': beta, 
                            'W0': W0, 
                            'log_random':log_random})
    return pd.DataFrame(records)

In [81]:
# Simulation parameters
np.random.seed(40)
n_surnames = 100
surnames = np.arange(n_surnames)

times = range(0, 21)
high_initial_wealth = np.random.lognormal(mean=10, sigma=2, size=n_surnames//2) 
low_initial_wealth = np.random.lognormal(mean=-10, sigma=2, size=n_surnames//2)
initial_wealth = np.concatenate([high_initial_wealth, low_initial_wealth])

high = np.full(n_surnames//2, 'high') 
low = np.full(n_surnames//2, 'low')
groups = np.concatenate([high, low])

# Scenario 1 -----------------------------------------------------------------------------------
mu_values_1 = np.random.normal(5, 5, size = n_surnames)
beta_values_1 = np.random.uniform(0.25, 0.35, size=n_surnames)  

# Population-wise convergence value
mu_population_1 = np.mean(mu_values_1)
W0_population_1 = np.mean(initial_wealth)
beta_population_1 = 0.3

# Generate Data for Scenario 1
df_scenario_1 = simulate_wealth(surnames, initial_wealth, mu_values_1, beta_values_1, times, groups)
population_data_1 = simulate_wealth(['Population'], [W0_population_1], [mu_population_1], [beta_population_1], times, groups)

# Scenario 2 -----------------------------------------------------------------------------------
mu_values_2 = np.full(n_surnames, 5)
beta_values_2 = np.random.uniform(0.75, 0.85, size=n_surnames)  

# Population-wise convergence value
mu_population_2 = 5
W0_population_2 = np.mean(initial_wealth)
beta_population_2 = 0.8

# Generate Data for Scenario 2
df_scenario_2 = simulate_wealth(surnames, initial_wealth, mu_values_2, beta_values_2, times, groups)
population_data_2 = simulate_wealth(['Population'], [W0_population_2], [mu_population_2], [beta_population_2], times, groups)

In [82]:
df_scenario_1

,Surname,Group,Time,Wealth,log_Wealth,mu,beta,W0,log_random
0,0,high,0,6534.849563,8.784905,5.171450,0.286882,6534.849563,-0.001584
1,0,high,1,496.748851,6.208085,5.171450,0.286882,6534.849563,-1.018887
2,0,high,2,237.185344,5.468842,5.171450,0.286882,6534.849563,0.644260
3,0,high,3,191.860132,5.256767,5.171450,0.286882,6534.849563,-0.294281
4,0,high,4,180.535259,5.195926,5.171450,0.286882,6534.849563,1.448112
...,...,...,...,...,...,...,...,...,...
2095,99,low,16,0.111704,-2.191902,-2.191902,0.271663,0.001415,-0.569793
2096,99,low,17,0.111704,-2.191902,-2.191902,0.271663,0.001415,-0.936125
2097,99,low,18,0.111704,-2.191902,-2.191902,0.271663,0.001415,-1.948268
2098,99,low,19,0.111704,-2.191902,-2.191902,0.271663,0.001415,0.683851


In [83]:
population_data_2

,Surname,Group,Time,Wealth,log_Wealth,mu,beta,W0,log_random
0,Population,high,0,52958.112613,10.877257,5,0.8,52958.112613,-1.757845
1,Population,high,1,16347.091026,9.701805,5,0.8,52958.112613,0.696411
2,Population,high,2,6383.323678,8.761444,5,0.8,52958.112613,0.217361
3,Population,high,3,3008.375029,8.009155,5,0.8,52958.112613,0.275378
4,Population,high,4,1648.010833,7.407324,5,0.8,52958.112613,0.049203
5,Population,high,5,1018.269022,6.925859,5,0.8,52958.112613,0.091339
6,Population,high,6,692.762717,6.540688,5,0.8,52958.112613,-0.620434
7,Population,high,7,509.051929,6.232550,5,0.8,52958.112613,-0.652639
8,Population,high,8,397.836066,5.986040,5,0.8,52958.112613,0.121815
9,Population,high,9,326.631303,5.788832,5,0.8,52958.112613,0.773866


In [84]:
color_start ='gray' #'#636EFA'
color_end ='salmon' #'#EF553B'
top_setting = dict(color = color_start, symbol = 'circle', size = 8)
bottom_setting = dict(color = color_end, symbol = 'x', size = 8)

fig = go.Figure()

# Add traces for 'high' group to color them differently
for i in df_scenario_1[df_scenario_1['Group'] == 'high']['Surname'].unique():
    df_nid = df_scenario_1[df_scenario_1['Surname'] == i]
    fig.add_trace(go.Scatter(x=df_nid['Time'], 
                             y=df_nid['log_Wealth']+df_nid['log_random'], 
                             mode='markers+lines', 
                             name=f'Surname {i}', 
                             opacity=0.4, 
                             line=dict(color= color_start, width=1.5),
                             marker = top_setting)) 


# Add traces for 'low' group 
for i in df_scenario_1[df_scenario_1['Group'] == 'low']['Surname'].unique():
    df_nid = df_scenario_1[df_scenario_1['Surname'] == i]
    fig.add_trace(go.Scatter(x=df_nid['Time'], 
                             y=df_nid['log_Wealth']+df_nid['log_random'], 
                             mode='markers+lines', 
                             name=f'Surname {i}', 
                             opacity=0.4, 
                             line=dict(color= color_end, width=1.5),
                             marker=bottom_setting)) 
                            


fig.add_trace(go.Scatter(x=population_data_1['Time'], y=population_data_1['mu'],
                         mode='lines', name='mu', line=dict(color='black', dash='dash', width=2.5)))

fig.update_layout(
    # title=f'ln(Wealth) over time: surname-specific functions in grey and population-wise convergence value {round(mu_population_1,2)} in red',
                  xaxis_title='Time',
                  yaxis_title='ln(Wealth)',
                  legend_title='Surnames',
                  font_family="Times New Roman",
                  title_font_family="Times New Roman",
                  font_size=24,
                  font_color='black',
                  # paper_bgcolor='rgba(0,0,0,0)',
                  plot_bgcolor = 'rgba(0,0,0,0)', 
                  showlegend=False,
                  margin=dict(l=10, r=10, t=10, b=10)
                 )

fig.show()

In [80]:
fig.write_image(path+"/img_revised/section1_a2.pdf")

In [74]:
fig = go.Figure()

# Add traces for 'high' group to color them differently
for i in df_scenario_2[df_scenario_2['Group'] == 'high']['Surname'].unique():
    df_nid = df_scenario_2[df_scenario_2['Surname'] == i]
    fig.add_trace(go.Scatter(x=df_nid['Time'], 
                             y=df_nid['log_Wealth']+df_nid['log_random'], 
                             mode='markers+lines', 
                             name=f'Surname {i}', 
                             opacity=0.4, 
                             line=dict(color=color_start, width=1.5),
                             marker=top_setting)) 

# Add traces for 'low' group 
for i in df_scenario_2[df_scenario_2['Group'] == 'low']['Surname'].unique():
    df_nid = df_scenario_2[df_scenario_2['Surname'] == i]
    fig.add_trace(go.Scatter(x=df_nid['Time'], 
                             y=df_nid['log_Wealth']+df_nid['log_random'], 
                             mode='markers+lines', 
                             name=f'Surname {i}', 
                             opacity=0.4, 
                             line=dict(color=color_end, width=1.5),
                             marker=bottom_setting))




fig.add_trace(go.Scatter(x=population_data_2['Time'], y=population_data_2['mu'],
                         mode='lines', name='mu', line=dict(color='black', dash='dash', width=2.5)))

fig.update_layout(
    # title=f'ln(Wealth) over time: surname-specific functions in grey and population-wise convergence value {round(mu_population_2,2)} in red',
                  xaxis_title='Time',
                  yaxis_title='ln(Wealth)',
                  legend_title='Surnames',
                  font_family="Times New Roman",
                  title_font_family="Times New Roman",
                  font_size = 24,
                  font_color='black',
                  # paper_bgcolor='rgba(0,0,0,0)',
                  plot_bgcolor = 'rgba(0,0,0,0)', 
                  showlegend=False,
                  margin=dict(l=10, r=10, t=10, b=10)
                 )

fig.show()

In [75]:
fig.write_image(path+"/img_revised/section1_b.pdf")

# Figure 2. The role of $\mu$ and $\beta$.

In [28]:
import plotly.colors

# Parameters (Fixed)
beta = 0.2
W0 = 1000
t = np.arange(0, 20)
mu_values = np.arange(1, 10)

color_start = '#636EFA'
color_end = '#EF553B'  

# Generate a continuous color scale from the two extreme colors defined above
colors = plotly.colors.sample_colorscale(
    plotly.colors.make_colorscale([color_start, color_end]), len(mu_values)
)


fig = go.Figure()

for i, mu in enumerate(mu_values):
    # Wealth calculation for each mu
    exp_part = (1 - beta**t) * mu
    wealth = np.exp(exp_part) * W0**(beta**t)
    
    fig.add_trace(go.Scatter(x=t, y=np.log(wealth), 
                            #  mode='lines+markers', 
                            mode='lines',
                             name=f'mu = {mu}',
                             line=dict(color=colors[i]),
                             marker=dict(size=0,
                                         colorscale=colors,
                                         colorbar=dict(title='mu', thickness=20,tickvals=[1, 9], ticktext=[1, 9])
                                         ),
                                showlegend=False
                             ))

fig.update_layout(
    xaxis=dict(title='Time'),
    yaxis=dict(title='ln(Wealth)'),
    # showlegend=True,
    font_family="Times New Roman",
    title_font_family="Times New Roman",
    font_size=24, 
    font_color='black',
    # paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor = 'rgba(0,0,0,0)',
    margin=dict(l=10, r=10, t=10, b=10)
)

fig.show()


In [29]:
fig.write_image("role_of_mu.pdf")

In [26]:
import plotly.colors

# Parameters (Fixed)
beta_values = np.arange(0.1, 1.0, 0.1) #spacing = 0.1
W0 = 1000
t = np.arange(0, 20)
mu = 9


color_start = '#636EFA'
color_end = '#EF553B'

# Generate a continuous color scale from the two extreme colors defined above
colors = plotly.colors.sample_colorscale(
    plotly.colors.make_colorscale([color_start, color_end]), len(beta_values)
)


fig = go.Figure()

for i, beta in enumerate(beta_values):
    # Wealth calculation for each beta
    exp_part = (1 - beta**t) * mu
    wealth = np.exp(exp_part) * W0**(beta**t)
    
    fig.add_trace(go.Scatter(x=t, y=np.log(wealth), 
                            mode='lines',
                             name=f'beta = {beta}',
                             line=dict(color=colors[i]),
                                showlegend=False
                             ))

fig.update_layout(
    xaxis=dict(title='Time'),
    yaxis=dict(title='ln(Wealth)'),
    # showlegend=True,
    font_family="Times New Roman",
    title_font_family="Times New Roman",
    font_size=24,
    font_color='black',
    # paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor = 'rgba(0,0,0,0)',
    margin=dict(l=10, r=10, t=10, b=10)
)

fig.show()


In [27]:
fig.write_image("role_of_beta.pdf")